<a href="https://colab.research.google.com/github/diyosukma/CP68-SistemRekomendasi/blob/main/Rekomendasi_Ekstrakurikuler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Laporan Capstone Project Team 68 - Rekomendasi Ekstrakurikuler

## Tema Proyek - Resource Sharing

## Business Understanding
Aplikasi ini dibuat untuk membantu para siswa SMA dalam menentukan kegiatan ekstrakurikuler. Sistem rekomendasi dimodelkan dengan metode Content-based Filtering, berdasarkan pengalaman-pengalaman para siswa SMA yang dulu pernah atau sekarang sedang mengikuti ekstrakurikuler. Sistem ini dibuat berdasarkan beberapa faktor yang berpengaruh.

### Latar Belakang
Peraturan [Permendikbud nomor 62 tahun 2014](https://jdih.kemdikbud.go.id/arsip/Permendikbud%20Nomor%2062%20Tahun%202014.pdf) tentang kegiatan ekstrakurikuler, menyebutkan bahwa setiap peserta didik memiliki hak mengikuti kegiatan ekstrakurikuler sesuai dengan minat dan bakat mereka. Penentuan pemilihan ekstrakurikuler berdasarkan minat dan bakat terjadi di Sekolah Menengah Atas (SMA) diserahkan sepenuhnya pada siswa. Menurut [Sari dan Simanjuntak (2020)](http://ejournal.upbatam.ac.id/index.php/comasiejournal/article/view/2141/1249) terkadang siswa cenderung bingung memilih ekstrakurikuler dikarenakan orang tua yang tidak suka dengan hobi anaknya atau siswa tersebut menganggap bahwa dirinya tidak sungguh-sungguh dengan hobi dan bakatnya. Maka dari itu dibangun sistem rekomendasi ekstrakurikuler pada siswa SMA dengan harapan membantu para siswa memilih ekstrakurikuler yang sesuai dengan mereka.

### Problem Statement
Bagaimana cara menentukan ekstrakurikuler yang sesuai dengan minat dan keadaan siswa SMA?

### Research Questions
Bagaimana cara membuat sistem rekomendasi ekstrakurikuler untuk anak SMA yang baik dan benar?
Apakah Metode Content-Based Filtering cocok digunakan untuk membuat sistem rekomendasi ekstrakurikuler ini?
Bagaimana hasil akurasi sistem rekomendasi yang dibuat?
Faktor apa yang berpengaruh dalam penentuan ekstrakurikuler siswa SMA?


## Data Understanding

Library yang dibutuhkan

In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from pathlib import Path
import matplotlib.pyplot as plt

### Data Loading

File yang akan digunakan adalah data hasil kuesioner mengenai hubungan ekstrakurikuler dan minat/bakat.

In [4]:
import pandas as pd
df = pd.read_csv('ekstrakurikuler.csv')
df

,Timestamp,Nama Lengkap,Asal Sekolah,Sulit Memilih,Gender,Agama,Jurusan,Bidang Ekstrakurikuler,Pekerjaan Ibu,Pekerjaan Bapak,Sungguh Berkegiatan,Minat Bakat Anda Tersalurkan,Alasan memilih,Teman berpegaruh,Lama pertemuan,Banyak pertemuan,Minat dan Bakat yang dimiliki
0,25/11/2021 7:21:17,Fahmi Nurfalah,SMA Negeri 5 Sukabumi,Tidak,Laki-laki,Islam,IPS,"Bidang Olahraga (seperti basket, futsal dll), ...",Ibu Rumah Tangga,Wiraswasta,Ya,Ya,Tekad dari diri sendiri,Ya,Lebih dari 1 Jam,2 Kali Seminggu,Cenderung menyukai untuk bekerja secara berkel...
1,25/11/2021 7:34:25,Nur Asiah Ramdani,SMA Negeri 1 Kota Sukabumi,Tidak,Perempuan,Islam,IPA,Bidang Akademik & Kebahasaan (seperti english ...,Ibu rumah tangga,PNS,Ya,Ya,Tekad dari diri sendiri,Ya,1 Jam,1 Kali Seminggu,"Menggemari kegiatan yang melibatkan tulisan, M..."
2,25/11/2021 9:13:46,Mojang Rucita Wijaksana,SMAN 1CIBADAK,Tidak,Perempuan,Islam,IPA,"Bidang Olahraga (seperti basket, futsal dll), ...",PNS,PNS,Ya,Ya,Tekad dari diri sendiri,Tidak,Lebih dari 1 Jam,2 Kali Seminggu,Mempertimbangkan baik sisi plus ataupun minus ...
3,25/11/2021 9:27:41,Wid Arfian,SMA 2 Sukabumi,Ya,Laki-laki,Islam,IPA,"Bidang Olahraga (seperti basket, futsal dll), ...",Wirausaha,Wirausaha,Ya,Ya,Pengaruh Teman,Ya,Lebih dari 1 Jam,1 Kali Seminggu,"Gemar berargumen, Memiliki peliharaan baik ber..."
4,25/11/2021 9:33:57,MEILAN WIDYANI NURHASANAH,SMAN 1 CIBADAK,Ya,Perempuan,Islam,IPA,Bidang Akademik & Kebahasaan (seperti english ...,Ibu rumah tangga,Karyawan swasta,Ya,Ya,Tekad dari diri sendiri,Tidak,Lebih dari 1 Jam,1 Kali Seminggu,Memahami sisi positif maupun negatif yang ada ...
5,25/11/2021 9:42:45,Ergina Kamilia Putri,SMAN 4 Kota Sukabumi,Ya,Perempuan,Islam,IPS,Bidang Akademik & Kebahasaan (seperti english ...,Ibu Rumah Tangga,PNS,Ya,Ya,Tekad dari diri sendiri,Ya,Lebih dari 1 Jam,2 Kali Seminggu,"Menggemari kegiatan yang melibatkan tulisan, G..."
6,25/11/2021 9:51:20,Aditya Cipta Rahadian,SMK Negeri 2 Kota Sukabumi,Tidak,Laki-laki,Islam,NaN,"Bidang Olahraga (seperti basket, futsal dll), ...",Wiraswasta,-,Ya,Ya,"Karena kebanyakan direkrut oleh teman, karena ...",Ya,Lebih dari 1 Jam,Lebih dari 3 Kali Seminggu,Gemar menuangkan ide dalam bentuk tulisan atau...
7,25/11/2021 9:53:09,Rivaldi Farhan,SMA N 3 Kota Sukabumi,Ya,Laki-laki,Islam,IPA,"Bidang Olahraga (seperti basket, futsal dll)",Ibu rumah tangga,Guru,Ya,Ya,Luka masa lalu,Tidak,Lebih dari 1 Jam,2 Kali Seminggu,Senang berinteraksi melalui verbal atau bercer...
8,25/11/2021 10:05:52,Diyo Sukma Pradana,SMAN 1 Cibadak,Ya,Laki-laki,Islam,IPA,Bidang Akademik & Kebahasaan (seperti english ...,Ibu Rumah Tangga,Pegawai Swasta,Ya,Ya,Tekad dari diri sendiri,Ya,Lebih dari 1 Jam,1 Kali Seminggu,"Menggemari kegiatan yang melibatkan tulisan, G..."
9,25/11/2021 10:10:14,Muhammad Abi,MAN Cibadak,Tidak,Laki-laki,Islam,IPA,"Bidang Olahraga (seperti basket, futsal dll)",Ibu Rumah Tangga,Wiraswasta,Ya,Ya,Tekad dari diri sendiri,Ya,Lebih dari 1 Jam,3 Kali Seminggu,Menyukai dan memiliki kemampuan dalam bermain ...


### Deskripsi Variabel

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59 entries, 0 to 58
Data columns (total 17 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   Timestamp                      59 non-null     object
 1   Nama Lengkap                   59 non-null     object
 2   Asal Sekolah                   59 non-null     object
 3   Sulit Memilih                  59 non-null     object
 4   Gender                         59 non-null     object
 5   Agama                          59 non-null     object
 6   Jurusan                        58 non-null     object
 7   Bidang Ekstrakurikuler         59 non-null     object
 8   Pekerjaan Ibu                  59 non-null     object
 9   Pekerjaan Bapak                59 non-null     object
 10  Sungguh Berkegiatan            59 non-null     object
 11  Minat Bakat Anda Tersalurkan   59 non-null     object
 12  Alasan memilih                 59 non-null     object
 13  Teman b

Fitur yang akan dipakai adalah Nama, sulit, gender, agama, jurusan, bidang, pekerjaan ibu dan bapak, sungguh, tersalurkan, teman, lama, banyak, minat dan bakat yang dimiliki

In [7]:
df = df[['Nama Lengkap', 'Sulit Memilih', 'Gender', 'Agama', 'Jurusan', 'Pekerjaan Ibu','Pekerjaan Bapak', 'Teman berpegaruh', 'Lama pertemuan', 'Banyak pertemuan', 'Minat dan Bakat yang dimiliki', 'Bidang Ekstrakurikuler']]
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59 entries, 0 to 58
Data columns (total 12 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   Nama Lengkap                   59 non-null     object
 1   Sulit Memilih                  59 non-null     object
 2   Gender                         59 non-null     object
 3   Agama                          59 non-null     object
 4   Jurusan                        58 non-null     object
 5   Pekerjaan Ibu                  59 non-null     object
 6   Pekerjaan Bapak                59 non-null     object
 7   Teman berpegaruh               59 non-null     object
 8   Lama pertemuan                 59 non-null     object
 9   Banyak pertemuan               59 non-null     object
 10  Minat dan Bakat yang dimiliki  59 non-null     object
 11  Bidang Ekstrakurikuler         59 non-null     object
dtypes: object(12)
memory usage: 5.7+ KB


In [15]:
df['Minat dan Bakat yang dimiliki'].unique()

array(['Cenderung menyukai untuk bekerja secara berkelompok, Menyukai dan memiliki kemampuan dalam bermain alat musik, Mampu dengan mudah meresapi nada atau melodi dari sebuah lagu, Gemar bekerja sambil mendendangkan suatu irama lagu atau musik, Menyukai kegiatan olah tubuh, Cenderung menggerakan anggota tubuh tertentu ketika berpikir, Sering merasa jenuh ketika diharuskan berdiam diri secara terus menerus, Lebih menyukai praktik daripada teori',
       'Menggemari kegiatan yang melibatkan tulisan, Memiliki ketertarikan dalam dunia menulis, khususnya menulis sebuah cerita fiksi, Cenderung memilih untuk melakukan berbagai hal secara individu, Cenderung menyukai bekerja hanya dengan kemampuan sendiri, Mempertimbangkan baik sisi plus ataupun minus terhadap keputusan yang akan diambil dalam menghadapi sebuah masalah diambil dalam menghadapi sebuah masalah, Suka bermain komputer dan menemukan cara kerja komputer',
       'Mempertimbangkan baik sisi plus ataupun minus terhadap keputusan yang

In [14]:
df.isnull().sum()

Nama Lengkap                     0
Sulit Memilih                    0
Gender                           0
Agama                            0
Jurusan                          1
Pekerjaan Ibu                    0
Pekerjaan Bapak                  0
Teman berpegaruh                 0
Lama pertemuan                   0
Banyak pertemuan                 0
Minat dan Bakat yang dimiliki    0
Bidang Ekstrakurikuler           0
dtype: int64